In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
import sys
from Bio import SeqIO
from data_generator import data_generator

#from google.colab import drive

In [2]:
large_file = "uniref50.fasta"
small_file = "100k_rows.fasta"

data_gen = data_generator(large_file, 1000)
data_size = 500
data_gen.gen_data(data_size)
print(len(data_gen.data))

500


In [3]:
class LSTM_model():
    def __init__(self, input_size, hidden_layer_size, nr_hidden_layers):
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.nr_hidden_layers = nr_hidden_layers
        
        self.model = nn.LSTM(input_size, hidden_layer_size, nr_hidden_layers).cuda()
        
        self.linear = nn.Linear(hidden_layer_size, input_size).cuda()
        
    def forward(self, input_data):
        lstm_out, _ = self.model(input_data)
        tag_space = self.linear(lstm_out)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [4]:
print(data_gen.data.shape)
seqs = data_gen.data.view(1000,data_size,21)
seqs = seqs.type(torch.float)
print(type(seqs))
seqs = seqs.to(device='cuda')
test = LSTM_model(21,70,1)


test_data = test.forward(seqs)


#output = test.forward(char_seqs)
#output = output.detach().numpy()
#argmax = np.argmax(output[0])
#print(argmax)
#print(output[0][argmax])

torch.Size([500, 1000, 21])
<class 'torch.Tensor'>


RuntimeError: CUDA out of memory. Tried to allocate 136.00 MiB (GPU 0; 1.96 GiB total capacity; 1.03 GiB already allocated; 89.00 MiB free; 1.09 GiB reserved in total by PyTorch)